In [22]:
import numpy as np

## Distance matrix between delivery points
# 0 being the warehouse and 1-6 the delivery points
#                    0   1   2   3   4   5   6
matrix = np.array([ [-1, 20, 18, 14, 16, 12, 19], # 0
                    [0, -1, 22, 18, 30, 26, 28],  # 1
                    [0, 0, -1, 32, 20, 22, 21],   # 2
                    [0, 0, 0, -1, 20, 22, 21],    # 3
                    [0, 0, 0, 0, -1, 30, 22],     # 4
                    [0, 0, 0, 0, 0, -1, 26],      # 5
                    [0, 0, 0, 0, 0, 0, -1]        # 6
                    ])

q = np.array([4, 6, 3, 5, 3, 6]) # Load for each delivery point

MAX_Q = 15 # Maximum units in truck



In [24]:
def fill_trucks(matrix, q, Q_max):

    def compute_savings(i, j, matrix=matrix):
        cost_independent_way = 2 * matrix[0][i] + 2 * matrix[0][j]
        cost_combined_way = matrix[0][i] + matrix[i][j] + matrix[0][j]
        savings = cost_independent_way - cost_combined_way
        # print(f'"Savings ({i},{j}) = {savings}"')
        return savings

    def get_all_savings(n, matrix=matrix):
        savings = {}
        for i in range(1, n+1):
            for j in range(i+1, n+1):
                savings[(i, j)] = compute_savings(i, j, matrix=matrix)
        savings = {k: v for k, v in sorted(savings.items(), key=lambda item: item[1], reverse=True)}
        return savings

    for i in range(matrix.shape[0]):
        for j in range(i):
            matrix[i][j] = matrix[j][i]

    # Check if the matrix is symmetric
    if not (matrix.transpose() == matrix).all():
        raise ValueError("Matrix must be symmetric")
    n = matrix.shape[0] -1 # number of nodes

    if len(q) != matrix.shape[1] - 1:
        raise ValueError("Matrix must be square")

    savings = get_all_savings(n, matrix=matrix)
    # print(savings)
    print("Savings:")
    for (i, j), saving in savings.items():
        print(f"({i}, {j}): {saving}")
    print("\n")

    trucks = [([], 0)] # (list of nodes/itinerary, load)
    nodes = {i: False for i in range(1, n+1)} # Dictionary to keep track of the nodes that have been added to a truck

    for (i, j), saving in savings.items(): # Go to the savings, from highest to lowest
        if nodes[i] and nodes[j]: # If both nodes are already in a truck, skip
            continue
        for idx, truck in enumerate(trucks):
            if i in truck[0] and truck[1] + q[j-1] <= MAX_Q and not nodes[j]:
                print(f"Truck {truck[0]} can take {j}, the load would be {truck[1] + q[j-1]}")
                truck[0].append(j)
                nodes[j] = True
                trucks[idx] = (truck[0], truck[1] + q[j-1])

            elif j in truck[0] and truck[1] + q[i-1] <= MAX_Q and not nodes[i]:
                print(f"Truck {truck[0]} can take {i}, the load would be {truck[1] + q[i-1]}")
                truck[0].append(i)
                nodes[i] = True
                trucks[idx] = (truck[0], truck[1] + q[i-1])
            else:
                if truck[1] + q[i-1] + q[j-1] <= MAX_Q:
                    print(f"Truck {truck[0]} can take {i} and {j}, the load would be {truck[1] + q[i-1] + q[j-1]}")
                    if i not in truck[0]:
                        truck[0].append(i)
                        nodes[i] = True
                    if j not in truck[0]:
                        truck[0].append(j) # Add the nodes to the truck
                        nodes[j] = True
                    trucks[idx] = (truck[0], truck[1] + q[i-1] + q[j-1]) # Update the load
                    break
        else:
            if not nodes[i] and not nodes[j]:
                trucks.append(([i, j], q[i-1] + q[j-1])) # If no truck can take the nodes, create a new truck

    # Check whether all nodes have been assigned to a truck
    if not all(nodes.values()):
        nodes_not_in_truck = [k for k, v in nodes.items() if not v]
        print(f"Nodes not in a truck: {nodes_not_in_truck}")

    # Else, print the trucks
    print("\nSuccessfully filled the trucks")
    for idx, truck in enumerate(trucks):
        print(f"Truck {idx}: {truck[0]}")

    return trucks, nodes

trucks, nodes = fill_trucks(matrix, q, MAX_Q)

Savings:
(1, 2): 16
(1, 3): 16
(2, 6): 16
(2, 4): 14
(4, 6): 13
(3, 6): 12
(1, 6): 11
(3, 4): 10
(2, 5): 8
(1, 4): 6
(1, 5): 6
(5, 6): 5
(3, 5): 4
(2, 3): 0
(4, 5): -2


Truck [] can take 1 and 2, the load would be 10
Truck [1, 2] can take 3, the load would be 13
Truck [4, 6] can take 5, the load would be 14
Nodes not in a truck: [4, 6]

Successfully filled the trucks
Truck 0: [1, 2, 3]
Truck 1: [4, 6, 5]
